In [1]:
import pandas as pd
import numpy as np
import datetime 

In [153]:
pd.set_option('display.max_rows', 1000)
pd.options.display.max_columns = 1000

In [ ]:
# import xlrd

# *Data Pulls*

In [ ]:
# dict_df = pd.read_excel('C:/Users/shant/GNIZE/Study/Integrations_Model_Data.xls',
#                    sheet_name=['HubSpot Customer Base Metadata','HubSpot Customer Base Value','Integration Ecosystem'])

In [ ]:
# dict_df['HubSpot Customer Base Metadata'].info()
# Cannot use xlrd package to read excel file because a critical number of rows are deleted. Expected 250k+, only received 65k+ 

In [185]:
hcm = pd.read_csv('C:/Users/shant/GNIZE/Study/HubSpot_Customer_Base_Metadata.csv', header=0)
hcv = pd.read_csv('C:/Users/shant/GNIZE/Study/HubSpot_Customer_Base_Value.csv', header=0) 
ie = pd.read_csv('C:/Users/shant/GNIZE/Study/Integration_Ecosystem.csv', header=0)

##### changing datatypes 

In [186]:
# hcm['Customer_ID'] = hcm['Customer_ID'].astype(str)
hcm['Report Month'] = pd.to_datetime(hcm['Report Month'], format='%Y-%m').dt.to_period('m')
hcv['Report Month'] = pd.to_datetime(hcv['Report Month'], format='%Y-%m').dt.to_period('m')
ie['Report Month'] = pd.to_datetime(ie['Report Month'], format='%Y-%m').dt.to_period('m')

In [187]:
def remove_dollar_and_commas_and_convert_to_float(value):
    if isinstance(value, str) and any(char.isdigit() for char in value):
        # Remove dollar sign and commas and convert to float
        value_cleaned = value.replace('$', '').replace(',', '')
        return float(value_cleaned)
    elif isinstance(value, (int, float)):
        return float(value)
    else:
        return value

In [188]:
hcv = hcv.applymap(remove_dollar_and_commas_and_convert_to_float)
ie = ie.applymap(remove_dollar_and_commas_and_convert_to_float)

new columns for anaysis

In [189]:
ie['Integration Total MRR Increase'] = ie['Integration Total MRR End of Month'].subtract(ie['Integration Total MRR Beginning of Month'])

In [190]:
check= ie[ie['Integration Total MRR Increase'] < 0]

In [191]:
check

,Integration ID,Report Month,Integration Connect Partner? (Yes/No),Paid Portal Installed Count,Free Portal Installed Count,Total Installed Count,Integration Total MRR Beginning of Month,Integration Total MRR End of Month,Integration Total Cancellation Dollars,Integration Total MRR Increase
104,105.0,2019-01,Yes,26.0,8.0,34.0,38092.0,37816.0,0.0,-276.0
117,118.0,2019-01,Yes,1882.0,943.0,2825.0,2824724.0,2812074.0,-42117.0,-12650.0
147,148.0,2019-01,No,1.0,1.0,2.0,900.0,875.0,0.0,-25.0
195,196.0,2019-01,No,2.0,0.0,2.0,9803.0,9556.0,0.0,-247.0
233,234.0,2019-01,No,81.0,29.0,110.0,164620.0,157350.0,-4964.0,-7270.0
...,...,...,...,...,...,...,...,...,...,...
32680,3400.0,2018-08,No,1.0,0.0,1.0,350.0,263.0,0.0,-87.0
32712,2201.0,2018-08,Yes,281.0,40.0,321.0,424663.0,422127.0,-100.0,-2536.0
32713,1954.0,2018-08,Yes,34.0,2.0,36.0,55586.0,54680.0,0.0,-906.0
32730,3019.0,2018-08,No,1.0,0.0,1.0,299.0,279.0,0.0,-20.0


###### EDA

How have integrations trended over time?

In [216]:
ie.groupby(['Integration Connect Partner? (Yes/No)', 'Report Month']).agg({'Total Installed Count': ['sum','mean', 'min', 'max']})

Total Installed Count  \
                                                                     sum   
Integration Connect Partner? (Yes/No) Report Month                         
No                                    2018-08                    27446.0   
                                      2018-09                    29303.0   
                                      2018-10                    31503.0   
                                      2018-11                    34260.0   
                                      2018-12                    36496.0   
                                      2019-01                    38774.0   
Yes                                   2018-08                   127739.0   
                                      2018-09                   135699.0   
                                      2018-10                   145145.0   
                                      2018-11                   153568.0   
                                      2018-12                   161471.0   
                                      2019-01                   172868.0   

                                                                              
                                                          mean  min      max  
Integration Connect Partner? (Yes/No) Report Month                            
No                                    2018-08         6.054710  0.0   2048.0  
                                      2018-09         6.087038  0.0   2170.0  
                                      2018-10         6.181907  0.0   2321.0  
                                      2018-11         6.353858  0.0   2452.0  
                                      2018-12         6.470922  0.0   2557.0  
                                      2019-01         6.456953  0.0   2713.0  
Yes                                   2018-08       661.860104  0.0  33644.0  
                                      2018-09       681.904523  0.0  35407.0  
                                      2018-10       687.890995  0.0  37579.0  
                                      2018-11       717.607477  1.0  39894.0  
                                      2018-12       740.692661  1.0  42036.0  
                                      2019-01       748.346320  1.0  45045.0

In [214]:
iot=ie.groupby(['Report Month','Integration Connect Partner? (Yes/No)'])['Total Installed Count','Integration Total Cancellation Dollars','Integration Total MRR Increase'].mean() 
iot.reset_index()

C:\Users\shant\miniconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,Report Month,Integration Connect Partner? (Yes/No),Total Installed Count,Integration Total Cancellation Dollars,Integration Total MRR Increase
0,2018-08,No,6.054710,-30.313920,43.919700
1,2018-08,Yes,661.860104,-3482.082902,4537.523316
2,2018-09,No,6.087038,-48.184462,73.633153
3,2018-09,Yes,681.904523,-4408.979899,7868.688442
4,2018-10,No,6.181907,-30.577512,98.160322
5,2018-10,Yes,687.890995,-3545.360190,8607.928910
6,2018-11,No,6.353858,-33.733309,89.181751
7,2018-11,Yes,717.607477,-3510.355140,6485.985981
8,2018-12,No,6.470922,-61.666135,60.636170
9,2018-12,Yes,740.692661,-6042.311927,4721.509174


,Report Month,Integration Connect Partner? (Yes/No),Total Installed Count,Integration Total Cancellation Dollars,Integration Total MRR Increase
0,2018-08,No,6.054710,-30.313920,43.919700
1,2018-08,Yes,661.860104,-3482.082902,4537.523316
2,2018-09,No,6.087038,-48.184462,73.633153
3,2018-09,Yes,681.904523,-4408.979899,7868.688442
4,2018-10,No,6.181907,-30.577512,98.160322
5,2018-10,Yes,687.890995,-3545.360190,8607.928910
6,2018-11,No,6.353858,-33.733309,89.181751
7,2018-11,Yes,717.607477,-3510.355140,6485.985981
8,2018-12,No,6.470922,-61.666135,60.636170
9,2018-12,Yes,740.692661,-6042.311927,4721.509174


In [ ]:
ie.groupby('Integration Connect Partner? (Yes/No)')['Customer Segment'].value_counts(normalize=True,dropna=False)

In [201]:
ie['Integration Connect Partner? (Yes/No)'].value_counts()

No     31480
Yes     1266
Name: Integration Connect Partner? (Yes/No), dtype: int64

In [202]:
ie.head()

,Integration ID,Report Month,Integration Connect Partner? (Yes/No),Paid Portal Installed Count,Free Portal Installed Count,Total Installed Count,Integration Total MRR Beginning of Month,Integration Total MRR End of Month,Integration Total Cancellation Dollars,Integration Total MRR Increase
0,1.0,2019-01,No,11.0,2.0,13.0,18976.0,19062.0,0.0,86.0
1,2.0,2019-01,No,1.0,1.0,2.0,1600.0,1600.0,0.0,0.0
2,3.0,2019-01,No,1.0,0.0,1.0,3748.0,3748.0,0.0,0.0
3,4.0,2019-01,No,1.0,1.0,2.0,3750.0,3758.0,0.0,8.0
4,5.0,2019-01,No,0.0,3.0,3.0,0.0,0.0,0.0,0.0


In [228]:
pt=hcm['Product Tier'].value_counts(normalize=True,dropna=False)

In [238]:
pt = pt.reset_index()

In [240]:
pt.columns = ['Product Tier', 'Distibution']
pt

,Product Tier,Distibution
0,Pro,0.588005
1,Starter,0.145006
2,Enterprise,0.137279
3,Basic,0.087164
4,NaN,0.042546
5,Total Integrations per Tier,Product Tier Basic 47820 Enterprise ...


In [241]:
pt['Total Integrations per Tier']= hcm.groupby('Product Tier')['Number of Integrations'].sum()

In [243]:
pt

,Product Tier,Distibution,Total Integrations per Tier
0,Pro,0.588005,NaN
1,Starter,0.145006,NaN
2,Enterprise,0.137279,NaN
3,Basic,0.087164,NaN
4,NaN,0.042546,NaN
5,Total Integrations per Tier,Product Tier Basic 47820 Enterprise ...,NaN


In [128]:
hcm.groupby('Product Tier')['Customer Segment'].value_counts(normalize=True,dropna=False)

Product Tier  Customer Segment        
Basic         NAM Low-Touch               0.302503
              NAM Partner                 0.196032
              International Partner       0.144137
              International Direct        0.127480
              International Low-touch     0.115552
              NAM High-touch              0.106177
              NAM Direct                  0.005273
              NAM Unowned                 0.002637
              International High-touch    0.000209
Enterprise    NAM Direct                  0.446987
              International Direct        0.164488
              NAM High-touch              0.137835
              NAM Low-Touch               0.079427
              NAM Partner                 0.057956
              International Partner       0.046716
              International Low-touch     0.041321
              International High-touch    0.020169
              NAM Unowned                 0.005102
Pro           NAM Low-Touch               0

In [222]:
hcm.head()

,Customer_ID,Report Month,Customer Employee Size,Product Tier,Customer Segment,Number of Integrations
0,41533,2018-09,1 to 25,Pro,International Direct,5
1,41533,2018-10,1 to 25,Pro,International Direct,5
2,41533,2018-11,1 to 25,Pro,International Direct,5
3,41533,2019-01,1 to 25,Pro,International Direct,8
4,41533,2018-12,1 to 25,Pro,International Direct,8


In [ ]:
hcv.groupby('Product Tier')['Customer Segment'].value_counts(normalize=True,dropna=False)